In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from time import time
import os
import xmltodict
import json
import re
from requests import get
from bs4 import BeautifulSoup
from pathlib import Path
import platform

platform = platform.platform()
cwd = os.getcwd()
download_path = os.path.join(cwd,'CRiS Downloads')
#adjust this to fit your specific file structure 
parent = str(Path(cwd).parents[0]) 

In [ ]:
if "Darwin" in platform:
    chrome_driver = os.path.join(parent, 'Drivers', 'chromedriver')
elif "Windows" in platform:
    chrome_driver = os.path.join(parent, 'Drivers', 'chromedriver.exe')
else:
    print("No OS/Chromedriver match. OS: {}".format(platform))

In [3]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_path,
        'disk-cache-size': 4096}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=chrome_driver, options=chromeOptions)
driver.get('http://cris.nih.go.kr/cris/en/search/basic_search.jsp')
driver.find_element_by_xpath("//input[@alt='Search']").click()
sleep(3)
total_trials = (driver.find_element_by_xpath("//strong[@class='searchtbtnb']")).text
driver.find_element_by_xpath("//img[@alt='Download']").click()
sleep(2)
handles = driver.window_handles
driver.switch_to.window(handles[-1])
select = Select(driver.find_element_by_id('data_dtype'))
select.select_by_value('xml')
driver.find_element_by_xpath("//img[@alt='confirm']").click()
sleep(5)

dl_check = 0
start_time = time()
while dl_check == 0 and time() - start_time < 600:
    count = 0
    for file in os.listdir(download_path):
        if file.endswith(".xml"):
            count = 1
        else:
            count = 0
    if count == 1:
        dl_check = 1
        driver.quit()
    else:
        dl_check = 0
        sleep(5)

In [7]:
type(total_trials)

str

In [5]:
print(os.listdir(download_path))
file_name = ''
for file in os.listdir(download_path):
    if file.endswith('.xml'):
        file_name = file
print(file_name)

['.DS_Store', 'cris_research_2019-02-18_en.xml']
cris_research_2019-02-18_en.xml


In [6]:
#there's, as far as I can tell, a single instance of a problematic html element (&#1;) making the XML invalid. 
#Upon Investigation, it appears to be completely inconsequential so rather than messing around with encoding, 
#much easier to just delete that 1 instance

xmlstring = open(download_path + '/' + file_name, 'r').read()
xml_fixed = re.sub(r'&#1;', '', xmlstring)

In [9]:
cris_trials = []

trials = xmltodict.parse(xml_fixed)
for trial in trials['trials']['trial']:
    cris_trials.append(json.dumps(trial))

if len(cris_trials) == int(total_trials):
    print("CRIS Download/Parse Successful: {} records".format(total_trials))
else:
    print("Trial Differential: CRIS Contains {}, Scrape Contains {}".format(total_trials, len(cris_trials)))


Download Successful: 3497 records


In [ ]:
print(cris_trials[62])

In [ ]:
test = json.loads(cris_trials[62])
print(test)

In [ ]:
test_new = test['main']['trial_id']
print(test_new)

In [ ]:
#creates a csv with no headers and a single column with all trials in json in their own row.

import csv
from datetime import date

def cris_csv():
    with open('cris - ' + str(date.today()) + '.csv','w', newline = '')as cris_csv:
        writer=csv.writer(cris_csv)
        for val in cris_trials:
            writer.writerow([val])

#If you want a CSV run this function after the script runs

142
